[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/02_TVM_Tutorial_Relay.ipynb)

Please run the following block to ensure TVM is setup for *this notebook*, each notebook may have its own runtime.

In [57]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! bash /tvm/package.sh
    # Add TVM to the Python path.
    import sys
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
else:
    print("Notebook executing locally, skipping Colab setup ...")

Notebook executing locally, skipping Colab setup ...


# Relay: an Extensible Deep Learning IR

Last year TVM introduced Relay IR – a second generation high-level IR for deep learning. 

Relay's design comes from a simple insight that the critical difference between regular IRs
and deep learning IRs are the primitive values they manipulate. Relay is designed using well
known insights from the programming languages community coupled with TVM's existing 
infrastructure to provide state of the art performance. 

If you are familiar with ideas from programming languages or existing computation graph
representations we will connect Relay to your existing knowledge during this tutorial.

We will first cover the design of Relay, then elaborate on how one can use it to 
accomplish a wide variety of tasks. This piece of the tutorial focused directly 
on Relay but Relay will be present throughout all of the content today, and serves
as the interface layer to TVM.

...

In [58]:
import tvm
from tvm import relay
import tvm.relay.testing
from tvm.relay.expr_functor import ExprMutator
import torch
import torchvision
import onnx

## Language 

We will briefly introduce the concepts of Relay below before showing how to use Relay to accomplish specific tasks.
You can find a full language specification [here](https://docs.tvm.ai/langref/index.html).

### Variables 

In [59]:
# A single Relay variable, the string is just a hint
x = relay.var('x')

# A Relay variable with a different dtype, defaults to float32.
x = relay.var('x', dtype='int32')

# A Relay variable with a different shape.
x = relay.var('x', shape=(10, 1))

### Operators

Relay provides high performance operators defined in TVM that implement the primitive operations needed by deep learning applications. Operators can be applied to arguments just like regular Python or C++ functions. Common arithemetic operations are provided both via names and operator overloading.

Variables can be used to construct Relay *expressions* which replace the concept of graphs present in previous frameworks. A Relay expression can be viewed much like a graph with extra functionality as we will see as we go
forward.

In [60]:
w = relay.op.add(x, x)
print(w)

v0.0.1
free_var %x: Tensor[(10, 1), float32]
add(%x, %x)


In [61]:
z = x + x
print(z)

v0.0.1
free_var %x: Tensor[(10, 1), float32]
add(%x, %x)


### Functions

The fundamental packaging of computation in Relay is the function. A function is a combination of a set of inputs,
and a Relay expression. One view is a function is no different than the ones in programming languages today, and another is that it replaces named subgraphs.

In [62]:
f = relay.Function([x], z)
print(f)

v0.0.1
fn (%x: Tensor[(10, 1), float32]) {
  add(%x, %x)
}


### Module

Finally we can give functions a global name and package many of them together into a module. When we add a function to the module, it will be type checked before hand.

When we print the module you can see the program annotated with all type information. 

In [63]:
mod = relay.Module({})
fname = relay.GlobalVar('f')
mod[fname] = f

print(mod)

v0.0.1
def @f(%x: Tensor[(10, 1), float32]) -> Tensor[(10, 1), float32] {
  add(%x, %x) /* ty=Tensor[(10, 1), float32] */
}



## Frontends

Relay comes with a variety of frontends and supports most major frameworks including TensorFlow, PyTorch, MxNet, ONNX, Keras and Caffe2.

Below we provide a couple examples of using these frontends to import models into Relay.

You can find specific tutorials on deploying pretrained models below:
    - [ONNX](https://docs.tvm.ai/tutorials/frontend/from_onnx.html#sphx-glr-tutorials-frontend-from-onnx-py)
    - [TensorFlow](https://docs.tvm.ai/tutorials/frontend/from_tensorflow.html#sphx-glr-tutorials-frontend-from-tensorflow-py)
    - [Keras](https://docs.tvm.ai/tutorials/frontend/from_keras.html#sphx-glr-tutorials-frontend-from-keras-py)
    - [PyTorch](https://tvm.ai/2019/05/30/pytorch-frontend)
    - [Caffe2](https://docs.tvm.ai/tutorials/frontend/from_caffe2.html#sphx-glr-tutorials-frontend-from-caffe2-py)

In [66]:
torch_resnet18 = torchvision.models.resnet18()
dummy_input = torch.randn(10, 3, 224, 224)
torch.onnx.export(torch_resnet18, dummy_input, "resnet.onnx", verbose=True)
onnx_resnet18 = onnx.load('resnet.onnx')
func, params = relay.frontend.from_onnx(onnx_resnet18, shape={ '0': (10, 3, 224, 224) })
print(func)

graph(%0 : Float(10, 3, 224, 224)
      %1 : Float(64, 3, 7, 7)
      %2 : Float(64)
      %3 : Float(64)
      %4 : Float(64)
      %5 : Float(64)
      %6 : Long()
      %7 : Float(64, 64, 3, 3)
      %8 : Float(64)
      %9 : Float(64)
      %10 : Float(64)
      %11 : Float(64)
      %12 : Long()
      %13 : Float(64, 64, 3, 3)
      %14 : Float(64)
      %15 : Float(64)
      %16 : Float(64)
      %17 : Float(64)
      %18 : Long()
      %19 : Float(64, 64, 3, 3)
      %20 : Float(64)
      %21 : Float(64)
      %22 : Float(64)
      %23 : Float(64)
      %24 : Long()
      %25 : Float(64, 64, 3, 3)
      %26 : Float(64)
      %27 : Float(64)
      %28 : Float(64)
      %29 : Float(64)
      %30 : Long()
      %31 : Float(128, 64, 3, 3)
      %32 : Float(128)
      %33 : Float(128)
      %34 : Float(128)
      %35 : Float(128)
      %36 : Long()
      %37 : Float(128, 128, 3, 3)
      %38 : Float(128)
      %39 : Float(128)
      %40 : Float(128)
      %41 : Float(128)
      %42 :

TVMError: Traceback (most recent call last):
  [bt] (8) 9   libtvm.dylib                        0x0000000115285fdd tvm::relay::backend::RelayBuildModule::GetFunction(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, std::__1::shared_ptr<tvm::runtime::ModuleNode> const&)::'lambda1'(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const + 429
  [bt] (7) 8   libtvm.dylib                        0x00000001152861f2 tvm::relay::backend::RelayBuildModule::Build(tvm::relay::Function, tvm::Map<tvm::Integer, tvm::Target, void, void> const&, tvm::Target const&) + 130
  [bt] (6) 7   libtvm.dylib                        0x00000001152863f9 tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::relay::Function, std::__1::unordered_map<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, tvm::runtime::NDArray, std::__1::hash<std::__1::basic_string<char, std::__1:

## Text Format

## Executing Relay


## Pass Manager

Relay has a flexible and configurable pass manager with an elegant API which be used to easily compose and schedule pass pipelines. We believe an easy to configure pipeline is important to enable intelligent exploration between a variety of 


## Optimizations

Defining optimizations to transform your program is straight forward and easy to do in Relay.

For example let's define a constant evaluator for Relay.

## Quantization

## Heterogeneous Execution

Relay supports a high-level interface for scheduling computation across multiple heterogeneous devices. An interesting property of this pass is that it is not special, it is built using Relay's standard machinery for
passes. 

We implement this by using an annotation to mark which computations we would like to schedule on which device, 
and a pass inserts all the appropriate calls to synchronize memory across devices. 

The below pass uses this machinery to schedule all convolutions onto the GPU.

In [52]:
class ScheduleConv2d(ExprMutator):
    def __init__(self, device):
        self.device = device
        super().__init__()

    def visit_call(self, expr):
        visit = super().visit_call(expr)
        if expr.op == tvm.relay.op.get("nn.conv2d"):
            return relay.annotation.on_device(visit, self.device)
        else:
            return visit

def schedule_conv2d_on_gpu(expr):
    sched = ScheduleConv2d(tvm.gpu(0))
    return sched.visit(expr)

In [53]:
# TODO(@jroesch): use pass manager
resnet, params = relay.testing.resnet.get_workload()
print(resnet)
resnet = schedule_conv2d_on_gpu(resnet)
print(resnet)
resnet = relay.ir_pass.rewrite_annotated_ops(resnet, 0)
print(resnet)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)

## Virtual Machine

Relay supports three execution mechanisms, the VM provides the best balance between performance and expressivity out of them, but is also the newest and is still in alpha quality. We are working on shipping the remaining pieces of the VM over the next few weeks, with a beta version coming soon. 

Details about the VM can be found here..., but users can use the VM today from the high-level executor interface in Python, and directly in C++.


## Ahead of time compilation
A final example of how what 

## VTA
...